In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!curl -L "https://universe.roboflow.com/ds/R5o58yegwx?key=orhOAT4dBE" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: train/images/pagi_16112021_mp4-498_jpg.rf.e941e45da938985e0ea6473846b01df4.jpg  
 extracting: train/images/pagi_16112021_mp4-49_jpg.rf.1ebd6717089c199d37f358b3ba1acf9e.jpg  
 extracting: train/images/pagi_16112021_mp4-4_jpg.rf.67448ac10fe97f9cbfea896f5bd4cb5f.jpg  
 extracting: train/images/pagi_16112021_mp4-500_jpg.rf.e9577a1bbfa6063656b78f80e015c0db.jpg  
 extracting: train/images/pagi_16112021_mp4-501_jpg.rf.ec043bbc6ec5e58fc50ae78c7de1384d.jpg  
 extracting: train/images/pagi_16112021_mp4-502_jpg.rf.43d3ed3b64d01c46642905532a26263a.jpg  
 extracting: train/images/pagi_16112021_mp4-503_jpg.rf.fe545665e56d73879cd268e807a36de4.jpg  
 extracting: train/images/pagi_16112021_mp4-504_jpg.rf.73453b206efef863a28640547d930749.jpg  
 extracting: train/images/pagi_16112021_mp4-505_jpg.rf.e13968e2fc914c31d94ede709d91a7fd.jpg  
 extracting: train/images/pagi_16112021_mp4-506_jpg.rf.b3bc59fa0ccd981d6401b527a817332a.jpg  
 extracting: train/images/p

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.3/904.3 kB 42.7 MB/s eta 0:00:00


In [ ]:
import os

from ultralytics import YOLO

model = YOLO()

model.train(data="/content/data.yaml", epochs=200, batch=32)

In [ ]:
from google.colab import files

!zip -r /content/runs.zip /content/runs

files.download('/content/runs.zip')

# predict

In [ ]:
import cv2
import torch
from ultralytics import YOLO

# load model
model = YOLO('./runs/detect/train/weights/last.pt')

# load the video file
input_video_path = './test.mp4'
output_video_path = './result.mp4'

# open the video
video_capture = cv2.VideoCapture(input_video_path)

# get video properties
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

# define the codec and create VideoWriter object to save output video
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # codec
out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Iterate over each frame
frame_count = 0
while video_capture.isOpened():
  ret, frame = video_capture.read()
  if not ret:
    break

  # yolo object detection
  results = model(frame)[0]

  # iterate through the detections and draw bounding boxes
  for result in results.boxes.data.tolist():
    # Each detection in the format [x1, y1, x2, y2, conf, class]
    x1, y1, x2, y2, conf, cls = result[:6]
    label = f'{model.names[cls]} {conf:.2f}'

    # draw bounding box and label on the frame
    if conf > 0.5:
      cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 4) # bounding box

  out_video.write(frame)

  # print progress
  frame_count += 1
  print(f'Processed frame {frame_count}/{total_frames}')

video_capture.release()
out_video.release()
cv2.destoryAllWindows()

print(f'Output video saved to {output_video_path}')